---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [26]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    r = []
    df = pd.read_table('university_towns.txt', header=None)
    for line in df[0]:
        if '[ed' in line:
            # this is a state and we need to extract only what is before ['edit']
            current_state = line.split('[ed')[0]
        else:
            region_name = line.split(' (')[0]
            r.append([current_state, region_name])
    df1 = pd.DataFrame(r, columns=["State", "RegionName"])
    #inv_states = {v: k for k, v in states.items()}
    #df1 = df1.replace(inv_states)
    return df1
univ_towns = get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [25]:
GDP = pd.read_excel('gdplev.xls', skiprows = 0)
GDP.columns = ['1', '2', '3', '4', 'Quater', '6', 'GDP', '8']
GDP = GDP[['Quater', 'GDP']]
index0 = GDP[GDP['Quater'] == '2000q1'].index[0]
GDP = GDP.loc[index0:]
GDP['GDP_prev'] = GDP['GDP'].shift(+1)
GDP['GDP_next'] = GDP['GDP'].shift(-2)
GDP['diff_prev'] = GDP['GDP'] - GDP['GDP_prev']
GDP['diff_next'] = GDP['GDP_next'] - GDP['GDP']
GDP['GDP_decline'] = (GDP['diff_prev'] < 0) & (GDP['diff_next'] < 0)    
GDP['GDP_up'] = (GDP['diff_prev'] > 0) & (GDP['diff_next'] > 0) 

,Quater,GDP,GDP_prev,GDP_next,diff_prev,diff_next,GDP_decline,GDP_up
219,2000q1,12359.1,NaN,12607.7,NaN,248.6,False,False
220,2000q2,12592.5,12359.1,12679.3,233.4,86.8,False,True
221,2000q3,12607.7,12592.5,12643.3,15.2,35.6,False,True
222,2000q4,12679.3,12607.7,12710.3,71.6,31,False,True
223,2001q1,12643.3,12679.3,12670.1,-36,26.8,False,False
224,2001q2,12710.3,12643.3,12705.3,67,-5,False,False
225,2001q3,12670.1,12710.3,12822.3,-40.2,152.2,False,False
226,2001q4,12705.3,12670.1,12893,35.2,187.7,False,True
227,2002q1,12822.3,12705.3,12955.8,117,133.5,False,True
228,2002q2,12893,12822.3,12964,70.7,71,False,True


In [21]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    return GDP[GDP['GDP_decline'] == True].iloc[0].Quater
reces_start = get_recession_start()


'2008q3'

In [23]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    GDP1 = GDP.copy()
    index1 = GDP1[GDP1['Quater'] == get_recession_start()].index[0]
    GDP1 = GDP1.loc[index1:]
    return GDP1[GDP1['GDP_up'] == True].iloc[1].Quater
reces_end = get_recession_end()

'2009q4'

In [24]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    GDP1 = GDP.copy()
    index1 = GDP1[GDP1['Quater'] == get_recession_start()].index[0]
    index2 = GDP1[GDP1['Quater'] == get_recession_end()].index[0]
    GDP1 = GDP1.loc[index1:index2]
    # print(GDP1['GDP'].min())
    # print(GDP1['GDP'].idxmin())
    return GDP1.loc[GDP1['GDP'].idxmin()].Quater
reces_bottom = get_recession_bottom()

'2009q2'

In [8]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    df1 = df[['State', 'RegionName']]
    index0 = df.columns.get_loc('2000-01')
    cols_to_drop = list(df.columns[6:index0])
    df.drop(cols_to_drop, axis=1, inplace=True)
    index0 = df.columns.get_loc('2000-01')
    list_of_months = list(df.columns[index0:-2])
    list_of_quarters = list(GDP.Quater)
    df2 = pd.DataFrame({x : [] for x in list_of_quarters})
    for counter, index in enumerate(list_of_quarters):
        # print("{},{}".format(index, list_of_months[slice(3*counter,3*counter+3)]))
        sub_df = df[list_of_months[slice(3*counter,3*counter+3)]]
        df2[index] = sub_df.apply(np.mean, axis=1)
    df2['2016q3'] = df[['2016-07', '2016-08']].apply(np.mean, axis=1)
    df3 = pd.merge(df1, df2, how='inner', left_index=True, right_index=True)
    df3.set_index(['State', 'RegionName'], inplace=True)
    df3.rename(states, inplace=True)
    return df3

In [154]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    univ_towns = get_list_of_university_towns()
    housing_data = convert_housing_data_to_quarters()
    housing_data['diff_reces'] = housing_data[get_recession_start()] - housing_data[get_recession_bottom()]
    housing_data['price_decline'] = housing_data['diff_reces'] > 0
    I = housing_data.columns.get_loc(reces_start)
    housing_data['price_ratio'] = housing_data[housing_data.columns[I-1]] / housing_data[reces_bottom]  
    # 
    univ_towns = get_list_of_university_towns()
    univ_towns.set_index(['State', 'RegionName'], inplace = True)
    univ_towns['is_ut'] = True
    df = pd.merge(housing_data, univ_towns, how='left', left_index=True, right_index=True)
    df['is_ut'].fillna(False, inplace=True)
    univ_town_df = df[df['is_ut'] == True]
    non_univ_town_df = df[df['is_ut'] == False]
    # print(univ_town_df.shape)
    # print(non_univ_town_df.shape)
    a = ttest_ind(univ_town_df['price_ratio'].dropna(), non_univ_town_df['price_ratio'].dropna())
    different = a.pvalue < 0.01
    p = a.pvalue
    mean_univ = np.mean(univ_town_df['price_ratio'].dropna()) 
    mean_non_univ = np.mean(non_univ_town_df['price_ratio'].dropna()) 
    better = 'non-university town' if mean_univ > mean_non_univ  else 'university town'
    return (different, p, better)
run_ttest()

(True, 0.002724063704761164, 'university town')